In [1]:
'''
@author: Mahmoud Mohammadi
Data Set : https://www.kaggle.com/cityofLA/city-payroll-data 

'''    

import csv
import collections as c
import math 
import operator
import numpy as np
import pandas as pd
import collections as c
import scipy.spatial.distance as sc
import copy
import datetime

import re


In [2]:
basePath ="data/"
dataset ="LACity"
dataFile = basePath + "data.csv"

df = pd.read_csv(dataFile, low_memory=False)

df.head()

,Row ID,Year,Department Title,Payroll Department,Record Number,Job Class Title,Employment Type,Hourly or Event Rate,Projected Annual Salary,Q1 Payments,...,MOU Title,FMS Department,Job Class,Pay Grade,Average Health Cost,Average Dental Cost,Average Basic Life,Average Benefit Cost,Benefits Plan,Job Class Link
0,111391,2014,Water And Power (DWP),NaN,1412316577,Commercial Service Representative,Full Time,NaN,$70386.48,$16129.89,...,NaN,98,1230,NaN,$16842.08,$1282.61,$0.00,$18124.69,DWP,http://per.lacity.org/perspecs/1230.pdf
1,31732,2013,Police (LAPD),4301.0,432728338,Police Officer I,Full Time,$25.12,$52450.56,$11331.00,...,POLICE OFFICERS UNIT,70,2214,A,$11651.40,$898.08,$191.04,$12740.52,Police,http://per.lacity.org/perspecs/2214.pdf
2,27697,2013,Police (LAPD),4301.0,97182506,Police Officer II,Full Time,$42.77,$89303.76,$20036.32,...,POLICE OFFICERS UNIT,70,2214,2,$11651.40,$898.08,$191.04,$12740.52,Police,http://per.lacity.org/perspecs/2214.pdf
3,14136,2013,Harbor (Port of LA),3201.0,950136941,Senior Security Officer,Full Time,$28.75,$60028.96,$15793.88,...,SUPV BLUE COLLAR,42,3184,0,$10710.24,$405.24,$11.40,$11126.88,City,http://per.lacity.org/perspecs/3184.pdf
4,91896,2014,Public Works - Sanitation,7024.0,3230003445,Senior Clerk Typist,Full Time,$30.92,$64553.13,$14700.00,...,CLERICAL UNIT,82,1368,0,$11000.40,$409.68,$11.40,$11421.48,City,http://per.lacity.org/perspecs/1368.pdf


In [3]:
full_columns = ['Row ID', 'Year', 'Department Title', 'Payroll Department',
       'Record Number', 'Job Class Title', 'Employment Type',
       'Hourly or Event Rate', 'Projected Annual Salary', 'Q1 Payments',
       'Q2 Payments', 'Q3 Payments', 'Q4 Payments', 'Payments Over Base Pay',
       '% Over Base Pay', 'Total Payments', 'Base Pay', 'Permanent Bonus Pay',
       'Longevity Bonus Pay', 'Temporary Bonus Pay', 'Lump Sum Pay',
       'Overtime Pay', 'Other Pay & Adjustments',
       'Other Pay (Payroll Explorer)', 'MOU', 'MOU Title', 'FMS Department',
       'Job Class', 'Pay Grade', 'Average Health Cost', 'Average Dental Cost',
       'Average Basic Life', 'Average Benefit Cost', 'Benefits Plan',
       'Job Class Link']
selected_columns = [ 'Row ID', 'Year',  'Record Number', 
       'Projected Annual Salary', 'Q1 Payments',
       'Q2 Payments', 'Q3 Payments', 'Q4 Payments', 'Payments Over Base Pay',
       '% Over Base Pay', 'Total Payments', 'Base Pay', 'Permanent Bonus Pay',
       'Longevity Bonus Pay', 'Temporary Bonus Pay', 'Lump Sum Pay',
       'Overtime Pay', 'Other Pay & Adjustments',
       'Other Pay (Payroll Explorer)',  'FMS Department',
       'Job Class', 'Average Health Cost', 'Average Dental Cost',
       'Average Basic Life', 'Average Benefit Cost', 
       ]
len(selected_columns)

25

In [4]:
# Selecting 24 columns having numeric data
df = df[selected_columns]
df.head()

,Row ID,Year,Record Number,Projected Annual Salary,Q1 Payments,Q2 Payments,Q3 Payments,Q4 Payments,Payments Over Base Pay,% Over Base Pay,...,Lump Sum Pay,Overtime Pay,Other Pay & Adjustments,Other Pay (Payroll Explorer),FMS Department,Job Class,Average Health Cost,Average Dental Cost,Average Basic Life,Average Benefit Cost
0,111391,2014,1412316577,$70386.48,$16129.89,$17829.78,$17678.24,$22946.57,$14273.91,23.67%,...,NaN,$369.12,$13904.79,$13904.79,98,1230,$16842.08,$1282.61,$0.00,$18124.69
1,31732,2013,432728338,$52450.56,$11331.00,$13859.93,$11968.32,$14048.20,$1700.40,3.43%,...,$0.00,$11.13,$180.24,$1689.27,70,2214,$11651.40,$898.08,$191.04,$12740.52
2,27697,2013,97182506,$89303.76,$20036.32,$23479.20,$21153.60,$24360.49,$4120.20,4.85%,...,$0.00,$482.29,$164.40,$3637.91,70,2214,$11651.40,$898.08,$191.04,$12740.52
3,14136,2013,950136941,$60028.96,$15793.88,$18560.38,$16471.18,$21110.80,$14275.44,24.76%,...,$0.00,$11504.72,$250.00,$2770.72,42,3184,$10710.24,$405.24,$11.40,$11126.88
4,91896,2014,3230003445,$64553.13,$14700.00,$17313.10,$15257.17,$17313.10,$5191.62,8.74%,...,$0.00,$417.37,$0.00,$4774.25,82,1368,$11000.40,$409.68,$11.40,$11421.48


In [5]:
df.isnull().sum()

Row ID                              0
Year                                0
Record Number                       0
Projected Annual Salary             0
Q1 Payments                         0
Q2 Payments                         0
Q3 Payments                         0
Q4 Payments                         0
Payments Over Base Pay              0
% Over Base Pay                     0
Total Payments                      0
Base Pay                            0
Permanent Bonus Pay                 0
Longevity Bonus Pay                 0
Temporary Bonus Pay                 0
Lump Sum Pay                    42264
Overtime Pay                      434
Other Pay & Adjustments             0
Other Pay (Payroll Explorer)        0
FMS Department                      0
Job Class                           0
Average Health Cost                 0
Average Dental Cost                 0
Average Basic Life                  0
Average Benefit Cost                0
dtype: int64

In [6]:
#Finding columns having null vlaues  and replacing them with 0.0
#df2 = df.copy()
df.fillna(-1. , inplace= True)

In [7]:
# Replacing non numeric characters such as $ or E in cell values
import re
theregex = re.compile(r'[^\d.]+')

df = df.astype('str').applymap(lambda x: re.sub(r'[^\d.-]+', '', x))       

# Changing all columns' types to float
df = df.astype('float')

In [53]:
#df2.drop('Label', axis=1, inplace=True)

In [8]:
df.head()

,Row ID,Year,Record Number,Projected Annual Salary,Q1 Payments,Q2 Payments,Q3 Payments,Q4 Payments,Payments Over Base Pay,% Over Base Pay,...,Lump Sum Pay,Overtime Pay,Other Pay & Adjustments,Other Pay (Payroll Explorer),FMS Department,Job Class,Average Health Cost,Average Dental Cost,Average Basic Life,Average Benefit Cost
0,111391.0,2014.0,1.412317e+09,70386.48,16129.89,17829.78,17678.24,22946.57,14273.91,23.67,...,-1.0,369.12,13904.79,13904.79,98.0,1230.0,16842.08,1282.61,0.00,18124.69
1,31732.0,2013.0,4.327283e+08,52450.56,11331.00,13859.93,11968.32,14048.20,1700.40,3.43,...,0.0,11.13,180.24,1689.27,70.0,2214.0,11651.40,898.08,191.04,12740.52
2,27697.0,2013.0,9.718251e+07,89303.76,20036.32,23479.20,21153.60,24360.49,4120.20,4.85,...,0.0,482.29,164.40,3637.91,70.0,2214.0,11651.40,898.08,191.04,12740.52
3,14136.0,2013.0,9.501369e+08,60028.96,15793.88,18560.38,16471.18,21110.80,14275.44,24.76,...,0.0,11504.72,250.00,2770.72,42.0,3184.0,10710.24,405.24,11.40,11126.88
4,91896.0,2014.0,3.230003e+09,64553.13,14700.00,17313.10,15257.17,17313.10,5191.62,8.74,...,0.0,417.37,0.00,4774.25,82.0,1368.0,11000.40,409.68,11.40,11421.48


In [9]:
# Saving cleaned DataFrame for later use
df.to_pickle("data/" + dataset + "_cleaned.pickle")

In [10]:
# min_max is an array containing the Min and Max of each column in every row. 
# min_max[1,0] = Min(Col[1]) , min_max[1,1] = Max(Col[1]), min_max[2,0] = Min(Col[2]) , min_max[2,1] = Max(Col[2]) , ....

col_min = [ min(df[col]) for col in df.columns ]
col_max = [ max(df[col]) for col in df.columns ]
min_max= np.array([ (x,y) for x,y in zip(col_min, col_max)])


import csv
# Saving Min and Max of each Dataframe column in a CSV file: [Min Col 0, Max Col 0],...

with open("data/"+ dataset + "_minmax.csv", 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(min_max)



In [12]:
# Column-wise Normalizing of raw data in range of -1,1
from sklearn import preprocessing


min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1 , 1)) 

df_scaled = min_max_scaler.fit_transform(df)

df_normalized = df.copy()
df_normalized = pd.DataFrame(df_scaled)


In [13]:
# Finding Average value of candidate column for labeling the data

col =   'Total Payments'             
print (max(df[col]) , min(df[col]) , df[col].median() , df[col].mean() , len(df[col] > 5500))

x = np.array([500930.73999999999 ,-30728.27,54959.080000000002 , 63014.632040153258 
               ])
x= x.reshape(-1, 1)
ds =min_max_scaler.fit_transform(x)
ds

500930.74 -30728.27 54959.08 63014.63204015281 285008


array([[ 1.        ],
       [-1.        ],
       [-0.6776605 ],
       [-0.64735705]])

In [22]:
# Adding a new collumn as label of each record. Will be used for training 
# -0.64 is the normalized value of the mean of column 9

label = np.array([ 1. if x > -0.64 else 0. for x in df_normalized[10]])

In [23]:
from scipy.stats import itemfreq

itemfreq(label)

array([[  0.00000000e+00,   1.62690000e+05],
       [  1.00000000e+00,   1.22318000e+05]])

In [190]:
# Saving normalized data with the lable as the last column in a pickle file

#label.to_pickle("data/y_train_" + dataset + ".pickle")

In [28]:
# Reshaping each row of the DataFrame to 5*5 matrix
data_dim = 5
ar = np.array(df_normalized.values.tolist())
ar = ar.reshape(ar.shape[0] ,data_dim , -1)


In [29]:
ar.shape

(285008, 5, 5)

In [30]:
# Saving reshaped data to be loaded later
import pickle

with open("data/X_train_" + dataset + ".pickle", 'wb') as handle:
    pickle.dump(ar , handle, protocol=pickle.HIGHEST_PROTOCOL)
 
with open("data/y_train_" + dataset + ".pickle", 'wb') as handle2:
    pickle.dump(label, handle2, protocol=pickle.HIGHEST_PROTOCOL)
    
    

In [40]:
with open("data/y_train_" + dataset + ".pickle", 'rb') as handle:
    y = pickle.load(handle)

In [41]:
# Should be similar to saved one 
print( str( y.shape))
y[0:2] == label[0:2]

(285008,)


array([ True,  True], dtype=bool)

In [46]:
y = y.astype(np.int)
y_dim = 2
y_vec = np.zeros((len(y), y_dim), dtype=np.float)
print(y_vec.shape)
for i, l in enumerate(y):
    y_vec[i, y[i]] = 1.0

(285008, 2)


In [47]:
y_vec

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [220]:
idx= 0
batch_size = 64
train_size =np.inf 
batch_idxs = min(len(df_loaded), train_size) // batch_size # config.batch_size
grayscale = True
print(batch_idxs)

4453


In [221]:
c = 0
for idx in range(0, batch_idxs):
    batch = df_loaded[idx *  batch_size : (idx + 1) * batch_size]

    batch_images = np.array(batch).astype(np.float32)
    c =c + 1

In [222]:
c , batch_images.shape , len(df_loaded)

(4453, (64, 5, 5), 285008)

In [224]:
idx  = 4453
batch = df_loaded[idx *  batch_size : (idx + 1) * batch_size]

batch_images = np.array(batch).astype(np.float32)

batch_images.shape

(16, 5, 5)